In [1]:
import os
import time

import numpy as np
from efficientnet_pytorch import EfficientNet
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

In [2]:
# Setting
batch_size = 16
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

models_dir = '../models'

input_sizes = [67, ]
load_model_paths = ['Preprocessed v1 EfficientNet-B3 v1.pth', ]

model_name = 'efficientnet-b3'  # Backbone model
test_dir = '../data/preprocessed_test_0615'  # path to test
class_dict_path = '../data/training data dic.txt'  # path to 玉山給的 training data dict.txt

In [3]:
# opt = Options().parse()

In [4]:
with open(class_dict_path, encoding='utf-8') as f:
    wordset = f.read().split('\n')

In [5]:
dataloaders = []

for size in input_sizes:
    data_transform = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        transforms.Normalize([0.726, 0.686, 0.695], [0.205, 0.210, 0.183])
    ])

    image_dataset = datasets.ImageFolder(test_dir, data_transform)
    
    dataloaders.append(torch.utils.data.DataLoader(image_dataset,
                                                   batch_size=batch_size,
                                                   shuffle=False,
                                                   num_workers=6))

dataset_sizes = len(dataloaders[0].dataset)

class_names = dataloaders[0].dataset.classes

In [6]:
models = []

for path in load_model_paths:
    model = EfficientNet.from_pretrained(model_name, num_classes=len(class_names))
    model.to(device)
    model.load_state_dict(torch.load(os.path.join(models_dir, path)))
    model.eval()
    models.append(model)

Loaded pretrained weights for efficientnet-b3


In [7]:
# model_test = EfficientNet.from_pretrained(model_name, num_classes=len(class_names))

# model_test.to(device)

# model_test.load_state_dict(torch.load(model_path))

# model_test.eval()
# print()

In [8]:
with torch.no_grad():
    model_max_probs = []
    model_preds = []
    model_labels = []
    for dataloader, model in zip(dataloaders, models):
        max_probs = []
        preds = []
        labels = []
        for batch_inputs, batch_labels in dataloader:
            batch_inputs = batch_inputs.to(device)
            batch_labels = batch_labels.to(device)

            batch_outputs = model(batch_inputs)
            softmax = nn.Softmax(dim=1)
            batch_max_prods, batch_preds = torch.max(softmax(batch_outputs), 1)
            max_probs += batch_max_prods.cpu().tolist()
            preds += batch_preds.cpu().tolist()
            labels += batch_labels.cpu().tolist()
        model_max_probs.append(max_probs)
        model_preds.append(preds)
        model_labels.append(labels)

model_max_probs = np.array(model_max_probs)
model_preds = np.array(model_preds)
model_labels = np.array(model_labels)

In [9]:
threshold = [0, 0.3, 0.5, 0.7]
model_scores = []

for max_probs, preds, labels in zip(model_max_probs, model_preds, model_labels):
    scores = {t: {'isnull_f1': 0, 'macro_f1': 0, 'weighted_f1': 0, 'accuracy': 0} for t in threshold}
    for t in threshold:
        preds[max_probs < t] = 0
        scores[t]['isnull_f1'] = f1_score(labels, preds, labels=[0], average="macro")
        scores[t]['macro_f1'] = f1_score(labels, preds, average="macro")
        scores[t]['weighted_f1'] = f1_score(labels, preds, average="weighted")
        scores[t]['accuracy'] = accuracy_score(labels, preds)
    model_scores.append(scores)

In [10]:
rows = []

for path, scores in zip(load_model_paths, model_scores):
    row = []
    for metric in ['isnull_f1', 'macro_f1', 'weighted_f1', 'accuracy']:
        for t in threshold:
            row.append(scores[t][metric])
    rows.append(row)

In [11]:
import pandas as pd 

df = pd.DataFrame(rows)
df.to_csv('../output/evaluation.csv', encoding='utf-8')